## Import

In [1]:
import sys
sys.path.append(r'C:\Users\Marco\Documents\GitHub\GeoSpatial-analysis\facility-location-Bergen\src\facility_location_Bergen\custome_modules')

In [2]:
import warnings
from shapely.errors import ShapelyDeprecationWarning
# Ignore the ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

In [3]:
import numpy as np
import pandas as pd
import pickle as pkl
import networkx as nx
import geopandas as gpd
from copy import deepcopy
import plotly.express as px
import plotly.graph_objects as go
import cartopy.io.img_tiles as cimgt
from plotly.subplots import make_subplots
from convert_geometry import toExtremePoints
from shapely.geometry import Point, LineString, Polygon
from log import print_INFO_message_timestamp, print_INFO_message
from retrieve_global_parameters import (
    retrieve_gdf_average_path,
    retrieve_gdf_path
)


In [4]:
times = ["20_04_2023"]

In [5]:
root = r"C:/Users/Marco/Documents/GitHub/GeoSpatial-analysis/facility-location-Bergen/"
paths = {time: root+retrieve_gdf_path(time, True, False) for time in times}

In [6]:
gdfs = {}
for time in times:
    with open(paths[time], "rb") as f:
        gdfs[time] = pkl.load(f)

In [8]:
gdf = gdfs[times[0]]
gdf.head(4)

,sourceUpdated,api_call_time,geometry_length,description,length,currentFlow.speed,currentFlow.speedUncapped,currentFlow.freeFlow,currentFlow.jamFactor,currentFlow.confidence,currentFlow.traversability,geometry.type,currentFlow.subSegments,currentFlow.jamTendency,currentFlow.junctionTraversability,geometry.multi_line,geometry.extreme_point
_id,,,,,,,,,,,,,,,,,
6453c60a68f2b2bf1c94ab37,2023-04-20 12:58:09+00:00,2023-04-20 13:00:00+00:00,"[21.0, 60.0, 26.0, 93.0, 14.0, 17.0, 79.0, 64.0]",Herland,374.0,13.611112,13.611112,16.11111,1.4,0.74,open,GeometryCollection,NaN,NaN,NaN,"MULTILINESTRING ((5.53747 60.42335, 5.53757 60...","(POINT (5.53747 60.42335), POINT (5.53903 60.4..."
6453c60a68f2b2bf1c94af46,2023-04-20 13:03:09+00:00,2023-04-20 13:05:00+00:00,"[21.0, 60.0, 26.0, 93.0, 14.0, 17.0, 79.0, 64.0]",Herland,374.0,13.055556,13.055556,16.11111,1.7,0.72,open,GeometryCollection,NaN,NaN,NaN,"MULTILINESTRING ((5.53747 60.42335, 5.53757 60...","(POINT (5.53747 60.42335), POINT (5.53903 60.4..."
6453c60a68f2b2bf1c94b12a,2023-04-20 13:08:09+00:00,2023-04-20 13:10:00+00:00,"[21.0, 60.0, 26.0, 93.0, 14.0, 17.0, 79.0, 64.0]",Herland,374.0,13.055556,13.055556,16.11111,1.6,0.72,open,GeometryCollection,NaN,NaN,NaN,"MULTILINESTRING ((5.53747 60.42335, 5.53757 60...","(POINT (5.53747 60.42335), POINT (5.53903 60.4..."
6453c60a68f2b2bf1c94b64c,2023-04-20 13:13:09+00:00,2023-04-20 13:15:00+00:00,"[21.0, 60.0, 26.0, 93.0, 14.0, 17.0, 79.0, 64.0]",Herland,374.0,13.055556,13.055556,16.11111,1.6,0.71,open,GeometryCollection,NaN,NaN,NaN,"MULTILINESTRING ((5.53747 60.42335, 5.53757 60...","(POINT (5.53747 60.42335), POINT (5.53903 60.4..."


In [9]:
mapping = {}
for j, col in enumerate(gdf.columns):
    mapping[col] = j

In [10]:
gdf.groupby("api_call_time").count().iloc[:, 0]

api_call_time
2023-04-20 05:30:00+00:00    619
2023-04-20 05:35:00+00:00    618
2023-04-20 05:40:00+00:00    619
2023-04-20 05:45:00+00:00    620
2023-04-20 05:50:00+00:00    618
                            ... 
2023-04-20 14:40:00+00:00    622
2023-04-20 14:45:00+00:00    623
2023-04-20 14:50:00+00:00    623
2023-04-20 14:55:00+00:00    626
2023-04-20 15:00:00+00:00    624
Name: sourceUpdated, Length: 63, dtype: int64

In [35]:
for j, row in enumerate(gdf.values):
    lines = pd.Series(row[mapping["geometry.multi_line"]])
    for i, line in enumerate(lines):
        p0 = toExtremePoints(line)[0]
        p1 = toExtremePoints(line)[-1]
        if p0 == Point((5.31522, 60.39234)) or p1 == Point((5.31522, 60.39234)):
            print(j,lines)

27463 0       LINESTRING (5.31762 60.39184, 5.3172 60.39195)
1       LINESTRING (5.3172 60.39195, 5.31672 60.39208)
2    LINESTRING (5.31672 60.39208, 5.31664 60.3921,...
3    LINESTRING (5.31615 60.39221, 5.31597 60.39224...
dtype: object
27464 0       LINESTRING (5.31762 60.39184, 5.3172 60.39195)
1       LINESTRING (5.3172 60.39195, 5.31672 60.39208)
2    LINESTRING (5.31672 60.39208, 5.31664 60.3921,...
3    LINESTRING (5.31615 60.39221, 5.31597 60.39224...
dtype: object
27465 0       LINESTRING (5.31762 60.39184, 5.3172 60.39195)
1       LINESTRING (5.3172 60.39195, 5.31672 60.39208)
2    LINESTRING (5.31672 60.39208, 5.31664 60.3921,...
3    LINESTRING (5.31615 60.39221, 5.31597 60.39224...
dtype: object
27466 0       LINESTRING (5.31762 60.39184, 5.3172 60.39195)
1       LINESTRING (5.3172 60.39195, 5.31672 60.39208)
2    LINESTRING (5.31672 60.39208, 5.31664 60.3921,...
3    LINESTRING (5.31615 60.39221, 5.31597 60.39224...
dtype: object
27467 0       LINESTRING (5.31762 60.391